In [ ]:
import numpy as np 
import pandas as pd 
import os,gc,re
import warnings
import sys
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import pickle

In [ ]:
train = pd.read_csv("/kaggle/input/chaii-hindi-and-tamil-question-answering/train.csv")
test = pd.read_csv("/kaggle/input/chaii-hindi-and-tamil-question-answering/test.csv")
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
print("training data size ",train.shape[0])
print("testing data size ",test.shape[0])

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=train, x="language")
plt.show()

In [ ]:
plt.figure(figsize=(20,7))
sns.histplot(train,kde=True)
plt.show()

In [ ]:
def unicode_to_ascii(s):
    text = s.encode('utf-8').decode('utf-8')
    return text


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    w = w.strip()
    return w


questions = []
answers = []
for ques,ans in zip(train['question'],train['answer_text']):
    questions.append(preprocess_sentence(ques))
    answers.append(preprocess_sentence(ans))

In [ ]:
print(f"questions {questions[100]}")
print(f"answere {answers[100]}")

In [ ]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

# Define start and end token to indicate the start and end of a sentence
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

VOCAB_SIZE = tokenizer.vocab_size+2

In [ ]:
print(f"encoded question example {tokenizer.encode(questions[100])}")

In [ ]:
MAX_LENGTH = 1000

def tokennize_filter(inputs,outputs):
    tokenized_inputs,tokenized_output = [],[]
    for (sen1,sen2) in zip(inputs,outputs):
        sen1 = START_TOKEN + tokenizer.encode(sen1) + END_TOKEN
        sen2 = START_TOKEN + tokenizer.encode(sen2) + END_TOKEN
        if len(sen1)<=MAX_LENGTH and len(sen2) <=MAX_LENGTH:
            tokenized_inputs.append(sen1)
            tokenized_output.append(sen2)
            
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs,maxlen=MAX_LENGTH,padding='post')
    tokenized_output = tf.keras.preprocessing.sequence.pad_sequences(tokenized_output,maxlen=MAX_LENGTH,padding='post')
    return tokenized_inputs,tokenized_output

In [ ]:
questions,answers = tokennize_filter(questions,answers)

In [ ]:
print(f'Vocab Size {VOCAB_SIZE}')
print(f'Number of samples {len(questions)}')

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((
    {'inputs': questions,
    'dec_inputs':answers[:,:-1]},
    {'outputs':answers[:,1:]},))

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
BATCH_SIZE = 32
BUFFER_SIZE = 20000

In [ ]:
def attention(query,key,value,mask):
    qk_matmul = tf.matmul(query,key,transpose_b=True)
    depth= tf.cast(tf.shape(key)[-1],tf.float32)
    logits = qk_matmul/tf.math.sqrt(depth)

    if mask is not None:
        logits += (mask *-1e9)
        attention_weight = tf.nn.softmax(logits,axis=-1)
        output = tf.matmul(attention_weight,value)

    return output

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # scaled dot-product attention
    scaled_attention = attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # concatenation of heads
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # final linear layer
    outputs = self.dense(concat_attention)

    return outputs

In [ ]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self,postion,d_model):
    super(PositionalEncoding,self).__init__()
    self.pos_encoding = self.postional_encoding(postion,d_model)
  def get_angles(self,postion,i,d_model):
    angles = 1/tf.pow(10000,(2*(i//2))/tf.cast(d_model,dtype=tf.float32))
    return postion*angles
  def postional_encoding(self,postion,d_model):
    angle_rads = self.get_angles(
        postion = tf.range(postion,dtype=tf.float32)[:,tf.newaxis],
        i = tf.range(d_model,dtype=tf.float32)[tf.newaxis,:],
        d_model = d_model
    )
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

    def call(self,inputs):
      return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            "pos_encoding":self.pos_encoding
        })
        return config

In [ ]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=0.3)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def encoder(vocab_size,num_layers,units,d_model,num_heads,dropout,name='encoder'):
  inputs = tf.keras.Input(shape=(None,),name='inputs')
  padding_mask = tf.keras.Input(shape=(1,1,None),name='padding_mask')
  
  embeddings = tf.keras.layers.Embedding(vocab_size,d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model,tf.float32))
  embeddings = PositionalEncoding(vocab_size,d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])
  return tf.keras.Model(
      inputs=[inputs,padding_mask],outputs=outputs,name=name
  )

In [ ]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],outputs=outputs,name=name)

In [ ]:
def decoder(vocab_size,num_layers,units,d_model,num_heads,dropout,name='decoder'):
    
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(units=units,d_model=d_model,num_heads=num_heads,dropout=dropout,name='decoder_layer_{}'.format(i),)(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],outputs=outputs,name=name)

In [ ]:
def transformer(vocab_size,num_layers,units,d_model,num_heads,dropout,name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1, 1, None),name='enc_padding_mask')(inputs)

    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(create_look_ahead_mask,output_shape=(1, None, None),name='look_ahead_mask')(dec_inputs)

    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1, 1, None),name='dec_padding_mask')(inputs)

    enc_outputs = encoder(vocab_size=vocab_size,num_layers=num_layers,units=units,d_model=d_model,num_heads=num_heads,dropout=dropout)(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(vocab_size=vocab_size,num_layers=num_layers,units=units,d_model=d_model,num_heads=num_heads,dropout=dropout,)(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [ ]:
#choosing hyperparameter
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.2

model = transformer(vocab_size=VOCAB_SIZE,num_layers=NUM_LAYERS,units=UNITS,d_model=D_MODEL,num_heads=NUM_HEADS,dropout=DROPOUT)
model.summary()

In [ ]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    # ensure labels have shape (batch_size, MAX_LENGTH - 1)
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

In [ ]:
model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
model.fit(dataset, epochs=1000)

In [ ]:
def evaluate(sentence,model):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # concatenated the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)


def predict(sentence,model):
  prediction = evaluate(sentence,model)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
sub = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv')
sub.head()

In [ ]:
question = test["question"]
questions=[]
for ques in question:
    questions.append(preprocess_sentence(ques))

In [ ]:
test_preds = []
for idx in range(len(question)):
    pred = predict(question[idx],model)
    test_preds.append(pred)

In [ ]:
sub['PredictionString'] = test_preds
sub[['id','PredictionString']].head()

In [ ]:
sub[['id','PredictionString']].to_csv('submission.csv',index=False)